In [18]:
#pip install imblearn

In [19]:
# Import dependencies
import pandas as pd
import numpy as np
import time
import operator
from collections import Counter
from imblearn.combine import SMOTEENN
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
# Create DataFrames from CSV files
t0 = time.time()
#lyric_TF_df = pd.read_csv('../Data/lyric_TF.csv')
filtered_lyric_TF_df = pd.read_csv('../Data/filtered_lyric_TF.csv')
#lyric_TF_df = lyric_TF_df.drop(0)
filtered_lyric_TF_df = filtered_lyric_TF_df.drop([0,1])
#lyric_TF_df.index = lyric_TF_df.index - 1
filtered_lyric_TF_df.index = filtered_lyric_TF_df.index - 2
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
filtered_lyric_TF_df.head()

Run time: 53.479844093322754 seconds


,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,...,entirely,basket,car,shawn,nothingness,amused,corners,interlude,sting,axis
0,willow,Taylor Swift,pop,8.0,"['dance', 'pop']",0.392,0.574,7.0,1.0,0.1700,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,1.0,0.2840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,WITHOUT YOU,The Kid LAROI,pop,8.0,['australian'],0.662,0.413,0.0,1.0,0.0299,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Heat Waves,Glass Animals,pop,8.0,"['shiver', 'indietronica', 'gauze']",0.761,0.525,11.0,1.0,0.0944,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,r u ok,Tate McRae,pop,8.0,"['dance', 'pop', 'electropop', 'post-teen']",0.666,0.593,2.0,1.0,0.0373,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# Create a dictionary of category names and ids
cat_name_id = {}
category_list = ['blues', 'classical', 'country', 'funk', 'hiphop', 'indie_alt', 'jazz', 
                 'metal', 'pop', 'punk', 'rnb', 'rock', 'romance', 'soul']
for cat in category_list:
    cat_id = list(filtered_lyric_TF_df[filtered_lyric_TF_df['category_name']==cat]['category_id'])[0]
    cat_name_id[cat] = cat_id
    print(cat, cat_id)

blues 0.0
classical 1.0
country 2.0
funk 3.0
hiphop 4.0
indie_alt 5.0
jazz 6.0
metal 7.0
pop 8.0
punk 9.0
rnb 10.0
rock 11.0
romance 12.0
soul 13.0


In [4]:
# Create a genre lists column
t0 = time.time()
genre_lists = []
for index, row in filtered_lyric_TF_df.iterrows():
    genres = row['genre_list'].replace('[', '').replace(']', '').replace("'", '').split(', ')
    genre_lists.append(genres)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
len(genre_lists)

Run time: 46.41603207588196 seconds


8014

In [5]:
# Replace the genre_list column
filtered_lyric_TF_df['genre_list'] = genre_lists

In [6]:
# Print how many songs there are in each category
for cat in category_list:
    song_count = len(filtered_lyric_TF_df[filtered_lyric_TF_df['category_name']==cat])
    print(f'There are {song_count} songs in the {cat} category.')

There are 423 songs in the blues category.
There are 83 songs in the classical category.
There are 1257 songs in the country category.
There are 281 songs in the funk category.
There are 683 songs in the hiphop category.
There are 657 songs in the indie_alt category.
There are 196 songs in the jazz category.
There are 892 songs in the metal category.
There are 691 songs in the pop category.
There are 569 songs in the punk category.
There are 362 songs in the rnb category.
There are 1229 songs in the rock category.
There are 280 songs in the romance category.
There are 411 songs in the soul category.


In [7]:
# Get the most popular genres for each category
t0 = time.time()
category_genres = {}
for category in category_list:
    genres = []
    for index, row in filtered_lyric_TF_df[filtered_lyric_TF_df['category_name']==category].iterrows():
        genres.extend(row['genre_list'])
    unique_genres = list(set(genres))
    genre_counts = []
    for genre in unique_genres:
        genre_counts.append((genre, genres.count(genre)))
    category_genres[category] = sorted(genre_counts, key = lambda x: x[1], reverse=True)[0:5]
    print(category, category_genres[category])
t1 = time.time()
print(f'Run time: {t1-t0} seconds')

blues [('blues', 259), ('electric', 189), ('modern', 168), ('traditional', 145), ('classic', 106)]
classical [('dance', 12), ('classical', 12), ('operatic', 11), ('', 9), ('pop', 9)]
country [('country', 977), ('contemporary', 847), ('modern', 277), ('pop', 88), ('nashville', 78)]
funk [('funk', 133), ('soul', 118), ('motown', 97), ('quiet', 82), ('disco', 79)]
hiphop [('rap', 345), ('hip', 304), ('pop', 227), ('trap', 192), ('southern', 142)]
indie_alt [('indie', 290), ('alternative', 181), ('new', 155), ('modern', 148), ('art', 142)]
jazz [('vocal', 92), ('contemporary', 82), ('jazz', 77), ('adult', 63), ('neo', 38)]
metal [('metal', 366), ('alternative', 284), ('nu', 221), ('rock', 220), ('hard', 207)]
pop [('pop', 343), ('indie', 180), ('dance', 163), ('electropop', 129), ('post-teen', 128)]
punk [('pop', 195), ('punk', 138), ('skate', 94), ('alternative', 84), ('melodic', 72)]
rnb [('pop', 219), ('r&b', 213), ('urban', 200), ('dance', 169), ('hip', 136)]
rock [('rock', 638), ('mod

In [8]:
# Find indeces to keep and indeces to remove
t0 = time.time()
cat_genre_dict = {'pop': ['pop'], 
                  'hiphop': ['rap', 'hip', 'trap'],
                  'metal': ['metal'], 
                  'rock': ['classic'], 
                  'jazz': ['jazz'], 
                  'rnb': ['r&b'], 
                  'romance': ['soft'], 
                  'soul': ['soul'], 
                  'indie_alt': ['indie', 'alternative'],
                  'classical': ['classical', 'operatic', 'dance'], 
                  'blues': ['modern', 'blues'],
                  'funk': ['funk', 'disco'], 
                  'punk': ['punk'],
                  'country': ['country']} 
indeces_to_keep = []
indeces_to_remove = []
for index, row in filtered_lyric_TF_df.iterrows():
    keep = False
    category = row['category_name']
    for genre in cat_genre_dict[category]:
        if genre in row['genre_list']:
            keep = True
            break
    if keep == True:
        indeces_to_keep.append(index)
    else:
        indeces_to_remove.append(index)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
print(len(indeces_to_keep))
len(indeces_to_remove)

Run time: 48.19620895385742 seconds
3975


4039

In [9]:
# Create non destructive copy and remove the unnecessary indeces
t0 = time.time()
filtered_lyric_TF_df2 = filtered_lyric_TF_df.drop(indeces_to_remove)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
filtered_lyric_TF_df2.head()

Run time: 0.3242759704589844 seconds


,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,...,entirely,basket,car,shawn,nothingness,amused,corners,interlude,sting,axis
0,willow,Taylor Swift,pop,8.0,"[dance, pop]",0.392,0.574,7.0,1.0,0.1700,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"[indie, pop, electropop]",0.581,0.584,2.0,1.0,0.2840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,r u ok,Tate McRae,pop,8.0,"[dance, pop, electropop, post-teen]",0.666,0.593,2.0,1.0,0.0373,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,BOYSHIT,Madison Beer,pop,8.0,"[dance, pop, electropop, post-teen]",0.638,0.699,1.0,1.0,0.0892,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Safe With Me (with Audrey Mika),Gryffin,pop,8.0,"[dance, pop, tropical, edm]",0.497,0.757,11.0,1.0,0.1270,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Remove the romance, punk, and blues categories
filtered_lyric_TF_df2 = filtered_lyric_TF_df2[filtered_lyric_TF_df2['category_name']!='romance']
filtered_lyric_TF_df2 = filtered_lyric_TF_df2[filtered_lyric_TF_df2['category_name']!='punk']
filtered_lyric_TF_df2 = filtered_lyric_TF_df2[filtered_lyric_TF_df2['category_name']!='blues']
#filtered_lyric_TF_df2 = filtered_lyric_TF_df2[filtered_lyric_TF_df2['category_name']!='indie_alt']
len(filtered_lyric_TF_df2)

3444

In [11]:
# Print how many songs there are in each category 
category_list = ['blues', 'classical', 'country', 'funk', 'hiphop', 'indie_alt', 'jazz', 
                 'metal', 'pop', 'punk', 'rnb', 'rock', 'romance', 'soul']
for cat in category_list:
    song_count = len(filtered_lyric_TF_df2[filtered_lyric_TF_df2['category_name']==cat])
    print(f'There are {song_count} songs in the {cat} category.')

There are 0 songs in the blues category.
There are 31 songs in the classical category.
There are 977 songs in the country category.
There are 163 songs in the funk category.
There are 446 songs in the hiphop category.
There are 375 songs in the indie_alt category.
There are 77 songs in the jazz category.
There are 366 songs in the metal category.
There are 343 songs in the pop category.
There are 0 songs in the punk category.
There are 213 songs in the rnb category.
There are 245 songs in the rock category.
There are 0 songs in the romance category.
There are 208 songs in the soul category.


In [42]:
# Define the features set
X = filtered_lyric_TF_df2.copy()
X = X.drop(['song_name', 'artist_name', 'category_name', 'category_id', 'genre_list'], axis=1)
X.head()

,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,...,entirely,basket,car,shawn,nothingness,amused,corners,interlude,sting,axis
0,0.392,0.574,7.0,1.0,0.1700,0.8330,0.00179,0.1450,0.529,81.112,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.581,0.584,2.0,1.0,0.2840,0.0805,0.00000,0.3660,0.756,179.954,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.666,0.593,2.0,1.0,0.0373,0.3180,0.00000,0.4140,0.329,140.013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.638,0.699,1.0,1.0,0.0892,0.0203,0.00000,0.0879,0.570,121.122,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.497,0.757,11.0,1.0,0.1270,0.0449,0.00000,0.1190,0.507,169.902,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
# Define the target set
y = filtered_lyric_TF_df2['category_id'].ravel()
y[:5]

array([8., 8., 8., 8., 8.])

In [67]:
# Split into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2583, 155)
(861, 155)
(2583,)
(861,)


# Combination Sampling with SMOTEENN

In [68]:
# Perform combination sampling on the data
t0 = time.time()
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')

Run time: 0.8587279319763184 seconds


In [69]:
# Count the resampled classes
Counter(y_resampled)

Counter({1.0: 183,
         2.0: 12,
         3.0: 75,
         4.0: 63,
         5.0: 71,
         6.0: 60,
         7.0: 59,
         8.0: 76,
         10.0: 101,
         11.0: 70,
         13.0: 90})

# Fitting the Model

In [70]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

In [71]:
# Fit the model
t0 = time.time()
rf_model = rf_model.fit(X_resampled, y_resampled)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')

Run time: 1.985875129699707 seconds


# Making Predictions

In [72]:
# Make predictions using the testing data
predictions = rf_model.predict(X_test)
predictions

array([ 7., 10., 11.,  8., 13., 11., 11.,  4.,  6., 13.,  8.,  8., 11.,
        7., 13., 13.,  4.,  8., 13.,  5.,  4., 11., 11.,  7.,  3.,  7.,
        3.,  8.,  1., 13.,  8.,  7.,  3.,  8.,  6.,  8., 11.,  5.,  8.,
        8.,  3., 10.,  7., 13.,  7.,  8., 13.,  8.,  1.,  4.,  5., 11.,
        4.,  8., 10.,  7.,  5.,  3.,  4., 10.,  3., 13.,  3.,  8., 11.,
       11.,  4.,  8.,  5.,  7.,  5., 13.,  6., 13.,  1.,  3.,  5., 13.,
       11.,  4.,  8., 11.,  1.,  8.,  6.,  7., 13.,  4.,  7.,  7., 10.,
        8.,  5.,  8.,  8., 13., 13., 13.,  5.,  8.,  8.,  1.,  4., 11.,
        8.,  7., 13.,  7.,  7.,  4.,  8.,  4.,  5., 13.,  7.,  7.,  5.,
        8., 10.,  8.,  5., 13., 11.,  4.,  3.,  8.,  7.,  3.,  8., 10.,
       13.,  8.,  4.,  4., 11.,  5.,  1.,  3., 13.,  8.,  8.,  7.,  8.,
        8.,  4.,  7.,  3.,  3.,  4.,  4., 11., 11.,  7.,  5.,  4.,  8.,
       11.,  4.,  4., 10., 11., 11.,  4.,  5.,  7., 11.,  3.,  3.,  8.,
        5.,  7.,  8.,  5.,  6., 10.,  5.,  8.,  3., 13.,  7.,  8

# Model Evaluation

In [73]:
# Calculate the accuracy score
acc_score = accuracy_score(y_test, predictions)
print(f'Accuracy Score: {acc_score}')

Accuracy Score: 0.47038327526132406


In [74]:
# Calculate the feature importance in the Random Forest Model
importances = rf_model.feature_importances_
importances

array([0.05194653, 0.04701884, 0.03684677, 0.03106392, 0.03495054,
       0.0293633 , 0.01784693, 0.01813337, 0.01341816, 0.01533961,
       0.01151013, 0.01385076, 0.00930557, 0.01239762, 0.01089005,
       0.00943232, 0.01210754, 0.00997213, 0.00889651, 0.00828731,
       0.00878591, 0.00811868, 0.00920469, 0.01027738, 0.01188087,
       0.00885783, 0.01210296, 0.00566497, 0.00874089, 0.00769431,
       0.00996609, 0.00747959, 0.00527456, 0.00738266, 0.00816544,
       0.00500213, 0.00586081, 0.00720708, 0.00770024, 0.00763315,
       0.00496297, 0.0082146 , 0.00742964, 0.00517536, 0.00325404,
       0.00652861, 0.00719848, 0.00549427, 0.00513282, 0.0016288 ,
       0.00466867, 0.00545117, 0.00571514, 0.00378475, 0.00575707,
       0.00527001, 0.00320054, 0.00670834, 0.00726942, 0.00637876,
       0.00520084, 0.00345545, 0.00709444, 0.00796891, 0.00239743,
       0.00216347, 0.00270132, 0.00524621, 0.0042649 , 0.00303451,
       0.00401665, 0.00260683, 0.00506697, 0.0037555 , 0.00411

In [75]:
# Sort the features by importance
sorted_features = sorted(zip(importances, X.columns), reverse=True)
sorted_features

[(0.05194652910228188, 'audio_ft_duration_ms'),
 (0.04701883720080095, 'audio_ft_danceability'),
 (0.03684676596413773, 'audio_ft_acousticness'),
 (0.03495053608294262, 'audio_ft_valence'),
 (0.031063916575947908, 'audio_ft_energy'),
 (0.029363295931242577, 'audio_ft_instrumentalness'),
 (0.018133371380973987, 'audio_ft_tempo'),
 (0.017846928378837525, 'audio_ft_speechiness'),
 (0.015339611096475737, 'im'),
 (0.013850759424616036, 'baby'),
 (0.013418161223475129, 'know'),
 (0.01239762305420736, 'oh'),
 (0.01210754005863204, 'youre'),
 (0.012102955433200793, 'audio_ft_key'),
 (0.01188086541134169, 'niggas'),
 (0.01151012546120416, 'dont'),
 (0.01089004842850407, 'cause'),
 (0.0102773836484599, 'audio_ft_liveness'),
 (0.009972126109881652, 'love'),
 (0.00996608807804265, 'gon'),
 (0.009432321771860267, 'get'),
 (0.009305573707363037, 'aint'),
 (0.009204686635758451, 'need'),
 (0.008896510849947456, 'got'),
 (0.008857827545254217, 'girl'),
 (0.008785912657745941, '?'),
 (0.008740885025345

# Dropping Lower Ranked Features

In [76]:
# Create a dictionary of features and accuracy scores
features_accuracy = {}
feature_percents = [0.1, 0.2, 0.3, 0.4]
for percent in feature_percents:
    # Define the features and target sets
    important_features = []
    sum = 0
    count = 0
    while sum < percent:
        sum = sum + sorted_features[count][0]
        important_features.append(sorted_features[count][1])
        count+=1
    X = filtered_lyric_TF_df2[important_features]
    y = filtered_lyric_TF_df2['category_id'].ravel()
    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
    # Scale the data
    scaler = StandardScaler()
    X_scaler = scaler.fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    # Fit the random forest model
    rf_model = RandomForestClassifier(n_estimators=500, random_state=78)
    t0 = time.time()
    rf_model = rf_model.fit(X_train_scaled, y_train)
    t1 = time.time()
    print(f'Run time to fit model with {percent*100}% of total features: {t1-t0} seconds')
    # Make predictions
    predictions = rf_model.predict(X_test_scaled)
    acc_score = accuracy_score(y_test, predictions)
    features_accuracy[percent] = acc_score

Run time to fit model with 10.0% of total features: 2.8775527477264404 seconds
Run time to fit model with 20.0% of total features: 3.5071380138397217 seconds
Run time to fit model with 30.0% of total features: 3.163722038269043 seconds
Run time to fit model with 40.0% of total features: 3.288294792175293 seconds


In [77]:
# Print feature percents and accuracy scores
for percent in features_accuracy:
    acc_score = features_accuracy[percent]
    print(f'Accuracy Score for {percent*100}% of total features: {acc_score}')

Accuracy Score for 10.0% of total features: 0.42392566782810687
Accuracy Score for 20.0% of total features: 0.46806039488966317
Accuracy Score for 30.0% of total features: 0.5993031358885017
Accuracy Score for 40.0% of total features: 0.6260162601626016


In [78]:
# Get the highest accuracy score
best_percent = max(features_accuracy, key=features_accuracy.get)
best_acc = features_accuracy[best_percent]
print(f'The highest accuracy score of {best_acc} was achieved using {best_percent*100}% of all features.')

The highest accuracy score of 0.6260162601626016 was achieved using 40.0% of all features.


# Accuracy score for SMOTEENN
- The highest accuracy score of 0.6434378629500581 was achieved using 20.0% of all features.

# Accuracy scores with different data
- Original <br>
The highest accuracy score of 0.5296296296296297 was achieved using 30.0% of all features.
- Remove romance category <br>
The highest accuracy score of 0.5805843543826579 was achieved using 30.0% of all features.
- Remove punksteam genre from punk category <br>
The highest accuracy score of 0.5872865275142315 was achieved using 40.0% of all features.
<br> [0.2 - 0.5] <br>
- Remove electropop genre from pop category <br>
The highest accuracy score of 0.5916030534351145 was achieved using 20.0% of all features. 
- Remove motown genre from soul category <br>
The highest accuracy score of 0.6032660902977905 was achieved using 40.0% of all features.
- Add dance genre to classical category <br>
The highest accuracy score of 0.6120689655172413 was achieved using 20.0% of all features.
<br> [0.1 - 0.5] <br>
- Add trap genre to hiphop category <br>
The highest accuracy score of 0.6213408876298395 was achieved using 30.0% of all features.
- Add nashville genre to country category <br>
The highest accuracy score of 0.6053604436229205 was achieved using 40.0% of all features. ??
- Replace rock genre with classic <br>
The highest accuracy score of 0.639344262295082 was achieved using 20.0% of all features.
- Replace blues genre with classic <br>
The highest accuracy score of 0.650319829424307 was achieved using 20.0% of all features.
<br> [0.1 - 0.4] <br>
- Replace blues genre with modern <br>
The highest accuracy score of 0.6509433962264151 was achieved using 30.0% of all features.

<br>
The highest accuracy score of 0.6119402985074627 was achieved using 40.0% of all features.
<br>
- Remove punk category <br>
The highest accuracy score of 0.6245847176079734 was achieved using 20.0% of all features. <br>
- Add blues genre <br>
The highest accuracy score of 0.6446808510638298 was achieved using 20.0% of all features. <br>
- Remove blues category <br>
The highest accuracy score of 0.6515679442508711 was achieved using 20.0% of all features. <br>
- Remove indie category <br>
The highest accuracy score of 0.7044270833333334 was achieved using 30.0% of all features. <br>

